# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 25 2022 12:03PM,249344,10,Stone8692355,Acute Outpatient Solutions,Released
1,Oct 25 2022 11:59AM,249343,10,Stone8692352,Acute Outpatient Solutions,Released
2,Oct 25 2022 11:56AM,249342,10,PRF-40217,Bio-PRF,Released
3,Oct 25 2022 11:56AM,249342,10,PRF-40220,Bio-PRF,Executing
4,Oct 25 2022 11:56AM,249342,10,PRF-40226,Bio-PRF,Released
5,Oct 25 2022 11:56AM,249342,10,PRF-40233,Bio-PRF,Released
6,Oct 25 2022 11:56AM,249342,10,PRF-40236,Bio-PRF,Released
7,Oct 25 2022 11:23AM,249302,15,PNC398358A,"Person & Covey, Inc.",Released
8,Oct 25 2022 11:23AM,249302,15,PNC398368A,"Person & Covey, Inc.",Released
9,Oct 25 2022 11:23AM,249302,15,PNC398408A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,249340,Executing,1
17,249342,Executing,1
18,249342,Released,4
19,249343,Released,1
20,249344,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249333,NaN,15.0,1.0
249340,NaN,1.0,NaN
249342,NaN,1.0,4.0
249343,NaN,NaN,1.0
249344,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0.0,1.0,0.0
249293,10.0,26.0,26.0
249298,5.0,3.0,6.0
249299,0.0,7.0,36.0
249301,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249279,0,1,0
249293,10,26,26
249298,5,3,6
249299,0,7,36
249301,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,0,1,0
1,249293,10,26,26
2,249298,5,3,6
3,249299,0,7,36
4,249301,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249279,,1,
1,249293,10,26,26
2,249298,5,3,6
3,249299,,7,36
4,249301,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF
7,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc."
14,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation
30,Oct 25 2022 10:58AM,249340,10,"Virtue Rx, LLC"
31,Oct 25 2022 10:28AM,249331,10,Eywa Pharma Inc.
53,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.
54,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc."
55,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions,,,1
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions,,,1
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF,,1,4
3,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc.",,,7
4,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation,,15,1
5,Oct 25 2022 10:58AM,249340,10,"Virtue Rx, LLC",,1,
6,Oct 25 2022 10:28AM,249331,10,Eywa Pharma Inc.,,21,1
7,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,,1,
8,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc.",,,1
9,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC",,7,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions,1,,
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions,1,,
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF,4,1,
3,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc.",7,,
4,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation,1,15,
5,Oct 25 2022 10:58AM,249340,10,"Virtue Rx, LLC",,1,
6,Oct 25 2022 10:28AM,249331,10,Eywa Pharma Inc.,1,21,
7,Oct 25 2022 9:04AM,249315,20,Alumier Labs Inc.,,1,
8,Oct 24 2022 5:44PM,249301,21,"NBTY Global, Inc.",1,,
9,Oct 24 2022 5:38PM,249299,15,"Brookfield Pharmaceuticals, LLC",36,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions,1,,
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions,1,,
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF,4,1,
3,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc.",7,,
4,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation,1,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions,1.0,NaN,NaN
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions,1.0,NaN,NaN
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF,4.0,1.0,NaN
3,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc.",7.0,NaN,NaN
4,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation,1.0,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 25 2022 12:03PM,249344,10,Acute Outpatient Solutions,1.0,0.0,0.0
1,Oct 25 2022 11:59AM,249343,10,Acute Outpatient Solutions,1.0,0.0,0.0
2,Oct 25 2022 11:56AM,249342,10,Bio-PRF,4.0,1.0,0.0
3,Oct 25 2022 11:23AM,249302,15,"Person & Covey, Inc.",7.0,0.0,0.0
4,Oct 25 2022 10:58AM,249333,10,ISDIN Corporation,1.0,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1496033,8.0,38.0,0.0
12,249279,0.0,1.0,0.0
15,997192,75.0,36.0,15.0
20,249315,0.0,1.0,0.0
21,249301,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1496033,8.0,38.0,0.0
1,12,249279,0.0,1.0,0.0
2,15,997192,75.0,36.0,15.0
3,20,249315,0.0,1.0,0.0
4,21,249301,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,38.0,0.0
1,12,0.0,1.0,0.0
2,15,75.0,36.0,15.0
3,20,0.0,1.0,0.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,0.0
2,15,Released,75.0
3,20,Released,0.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,20,21
Status,,,,,
Completed,0.0,0.0,15.0,0.0,0.0
Executing,38.0,1.0,36.0,1.0,0.0
Released,8.0,0.0,75.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,20,21
0,Completed,0.0,0.0,15.0,0.0,0.0
1,Executing,38.0,1.0,36.0,1.0,0.0
2,Released,8.0,0.0,75.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,20,21
0,Completed,0.0,0.0,15.0,0.0,0.0
1,Executing,38.0,1.0,36.0,1.0,0.0
2,Released,8.0,0.0,75.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()